# Collect Reddit data

**Author:** Hunter Priniski (priniski@ucla.edu)
**Script:** Mine Reddit posts. 

In [ ]:
import praw
import json
import datetime
import os
from ahocorapy.keywordtree import KeywordTree
import pandas as pd
from nltk.stem.porter import *
import re

In [ ]:
reddit = praw.Reddit(client_id='bCQSZCUJwMbIWA',
                     client_secret='MMTmTsQyE3ZxniK9hsYU7QnQrJw',
                     password='TS26034ts',
                     user_agent='testscript by /u/hunugh',
                     username='hunugh')

In [ ]:
def get_replies(comment):
    replies = comment['_replies']._comments

    replies_list = []

    if len(replies) > 0:

        for reply in replies:
            replies_list.append(reply.id)

        return replies_list

    else:
        return []

In [ ]:
def get_comments(submission):
    
    comments = []
    com = submission.comments.replace_more(limit=submission.num_comments)
    com_tree = submission.comments[:]

    while com_tree:
        comment = com_tree.pop(0)                    
        com_tree.extend(comment.replies)
        comments.append(comment)

    data_dicts = []
    
    for comment in comments:
        comment_dict = vars(comment)
        comment_body = comment_dict['body']
        data_dict = {field:comment_dict[field] for field in comment_dict.keys()}
        data_dict['_replies'] = get_replies(comment_dict)
        
        try: 
            data_dict['author'] = vars(data_dict['author'])['name']
        
        except TypeError: 
            continue
            
        data_dicts.append(data_dict)

    return data_dicts

In [ ]:
subreddit_name = 'statistics'
subreddit = reddit.subreddit(subreddit_name)

for submission in subreddit.top(limit = 1000):

    submission_dict = vars(submission)
    author = submission_dict['author']
    post = {field:submission_dict[field] for field in submission_dict.keys()}
    post['_comments'] = get_comments(submission)
    if post['author']: 
        post['author'] = vars(post['author'])['name']
    else: 
        post['author'] = 'NA'
    post['_reddit'] = '_reddit'
    post['subreddit'] = subreddit_name
    
    for comment in post['_comments']:
        comment['_submission'] = comment['_submission'].id
        comment['_reddit'] = '_reddit'
        comment['subreddit'] = subreddit_name
        
    with open('reddit_data/'+ post['name'] +'.json', 'w') as outfile:
        print("Wrote %s as JSON File" % post['name'])
        json.dump(post, outfile)

In [ ]:
posts = [] 
#Note. you need to create a dir called reddit_data in your wd
directory = os.fsencode('reddit_data/')     

for file in os.listdir(directory): 
    filename = os.fsdecode(file)    
    if filename.endswith(".json"):  
        
        with open('reddit_data/' + filename, 'r') as json_file:
            post = json.load(json_file)
            posts.append(post)  

In [ ]:
df = []

#create a trim dataframe of reddit posts
for post in posts:
    subreddit = post['subreddit']
    title = post['title']
    selftext = post['selftext']
    selftext_html = post['selftext_html']
    name = post['name']
    author = post['author']
    score = post['score']
    
    
    for comment in post['_comments']:
        
        comment_replies = comment['_replies']
        comment_body = comment['body']
        comment_html = comment['body_html']    
        comment_name = comment['name']
        comment_author = comment['author']
        comment_score = comment['ups']
    
        replies = comment['_replies']
        
        row = [subreddit, title, selftext, selftext_html, name, author, score,
              comment_replies, comment_body, comment_html, comment_name, comment_author, comment_score]
        
        df.append(row)

In [ ]:
cols = ['subreddit', 'title', 'selftext', 'selftext_html', 'name', 'author', 'score',
              'comment_replies', 'comment_body', 'comment_html', 'comment_name', 'comment_author', 'comment_score']

In [ ]:
DF = pd.DataFrame(df, columns = cols)

In [ ]:
DF.to_excel('reddit_data.xlsx', index = False)